In [ ]:
"""
Spoofing is a form of market manipulation where a trader enters non-bona-fide orders on the opposite side of the order book from his bona-fide order to increase the perceived interest in the stock. Once filled on the bona-fide order the non-bona-fide orders are cancelled.
1.       This report should identify traders (group them if possible).
2.       The report should also have the min fields noted above
3.       I would like to identify when a trader has an existing position, marked long or short.
a.       And enters an order, or series of orders on the opposite side of the position for any size.  
b.      And when an execution is received that offsets the position, the order(s) on the contra side of the execution are cancelled within a two second timeframe.
c.       Any order or series of orders (layering), that are cancelled within a 2 second timeframe should be identified.
d.      Essentially, you want to identify an execution that flattens out a position, where orders were entered and subsequently cancelled on the opposite side of the executed order. 

"""


In [4]:
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import mysql.connector
import psycopg2
import ftplib
from ftplib import FTP_TLS
import ssl
import pandas as pd
import datetime
import numpy as np
import datetime
import shutil

In [6]:
localFilePath = '/home/elliott/development/files/'
changeFileName = 'change.csv'
positionsFileName = 'positions.csv'

d = datetime.datetime.combine(datetime.date.today(), datetime.time()) - datetime.timedelta(days=1)
d2 = d - datetime.timedelta(days=1)
d3 = d2 - datetime.timedelta(days=1)

# Open the local files
localFile = open(os.path.join(localFilePath, changeFileName), 'wb')
localFileP = open(os.path.join(localFilePath, positionsFileName), 'wb')

# Open the local change file and write its content to localFile
try:
    with open('Scalp_Trade_EXT982_' + d2.strftime('%Y%m%d') + ".csv", 'rb') as changeFile:
        localFile.write(changeFile.read())
except FileNotFoundError as e:
    print(f"Error: {changeFileName} not found in {localFilePath}")
    print(e)

# Open the local positions file and write its content to localFileP
try:
    with open('Scalp_Trade_EXT982_' + d2.strftime('%Y%m%d') + ".csv", 'rb') as positionsFile:
        localFileP.write(positionsFile.read())
except FileNotFoundError as e:
    print(f"Error: {positionsFileName} not found in {localFilePath}")
    print(e)

# Close the local files
localFile.close()
localFileP.close()


Error: change.csv not found in /home/elliott/development/files/
[Errno 2] No such file or directory: 'Scalp_Trade_EXT982_20230329.csv'
Error: positions.csv not found in /home/elliott/development/files/
[Errno 2] No such file or directory: 'Scalp_Trade_EXT982_20230329.csv'


In [9]:
sterling = pd.read_csv('/home/elliott/development/files/change.csv', sep=',', header=0)
positions = pd.read_csv('/home/elliott/development/files/positions.csv', sep=',', header=0)
fast = pd.read_csv('/home/elliott/development/files/fast.csv', sep=',', header=None, lineterminator='\n', low_memory=False)

In [18]:
sterling.head(300)

,AccountNumber,Firm,OfficeCode,StrategyId,StrategySequence,AccountType,CUSIP,TradeQuantity,Symbol,Description,...,Change,PositionType,ProcessDate,IsSelling,MaintenanceRequirement,ConcentrationRequirement,OptionStrategy,OptionLeg,StrategyMaintenanceRequirement,StrategyConcentrationRequirement
0,3OB05254,10,3OB,107,9,Margin,O02PCWSM6,3.0,VRTX 20240119P 100.000,PUT VRTX 01/19/24 100,...,0.0,Option,2023-03-28 00:00:00,0,0.0,0.0,Universal Spread,DD9,0.0,0.0
1,3OB05258,10,3OB,9,9,Margin,O02PCWSM6,3.0,VRTX 20240119P 100.000,PUT VRTX 01/19/24 100,...,0.0,Option,2023-03-28 00:00:00,0,0.0,0.0,Universal Spread,J9,0.0,0.0
2,3OB05254,10,3OB,127,8,Margin,O04NPZPG5,-16.0,FRC 20230331P 1.000,PUT FRC 03/31/23 1,...,0.0,Option,2023-03-28 00:00:00,1,0.0,0.0,Universal Spread,DX8,0.0,0.0
3,3OB05254,10,3OB,107,8,Margin,O01SDNPH6,-25.0,VRTX 20240119P 90.000,PUT VRTX 01/19/24 90,...,0.0,Option,2023-03-28 00:00:00,1,0.0,0.0,Universal Spread,DD8,0.0,0.0
4,3OB05254,10,3OB,81,8,Margin,O02X23443,-25.0,LLY 20230421C 500.000,CALL LLY 04/21/23 500,...,0.0,Option,2023-03-28 00:00:00,1,0.0,0.0,Universal Spread,CD8,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,3OB05255,10,3OB,6,1,Margin,O01SYSHJ7,20.0,BAC 20230421C 35.000,CALL BAC 04/21/23 35,...,0.0,Option,2023-03-28 00:00:00,0,0.0,0.0,Long Call,G1,0.0,0.0
296,3OB05255,10,3OB,7,1,Short,060505104,-2000.0,BAC,BANK OF AMERICA CORP,...,0.0,Equity,2023-03-28 00:00:00,1,28632.0,28632.0,Covered Put,H1,28632.0,28632.0
297,3OB05255,10,3OB,9,1,Margin,L0175J104,2500.0,ASPS,ALTISOURCE PORTFOLIO,...,0.0,Equity,2023-03-28 00:00:00,0,11850.0,11850.0,Equity,J1,11850.0,11850.0
298,3OB05255,10,3OB,10,1,Margin,O02LVXDZ8,10.0,YPF 20230406C 12.000,CALL YPF 04/06/23 12,...,0.0,Option,2023-03-28 00:00:00,0,0.0,0.0,Long Call,K1,0.0,0.0


In [12]:
# Initialize positionDict and symbolsList
positionDict = {}
symbolsList = []

# Loop through each row in the positions DataFrame
for x in range(len(positions)):
    # Extract symbol and trader information from relevant columns
    sym = positions['Symbol'][x].replace(" ", "")
    trader = positions['AccountNumber'][x]
    # Add information to positionDict using setdefault method
    positionDict.setdefault(trader[-4:], {}).setdefault(sym, positions['Symbol'][x])
    # Add symbol to symbolsList if it's not already in the list
    if sym not in symbolsList:
        symbolsList.append(sym)

# Create spoofs DataFrame with zeros
spoofs = pd.DataFrame(np.zeros((3500, 16)), columns=["Date", "Time", "Time Entered", "Side", "Symbol",
                                                     "Position", "Quantity", "Maturity", "Strike Price",
                                                     "Order Price", "Open Close", "Order Type", "OrderID",
                                                     "Status", "Exchange", "Account"])

sterLength = len(sterling)
sterlingTimes = sterling['Time']
# Loop through each row in the sterling DataFrame
for i in range(1, sterLength - 1):
    # Check if execution quantity is less than 1
    if sterling['Exe Qty'][i] < 1:
        continue
    
    # Extract various pieces of information from the row
    temp = i + 1
    if '#' in sterlingTimes[temp]:
        continue
    symbol = sterling['Symbol'][i]
    account = sterling['ID'][i]
    tradr = account[2:6]
    indx = symbol.find(' ')
    amt = sterling['Exe Qty'][i]
    spacedSymbol = symbol
    t1 = datetime.datetime.strptime(sterlingTimes[i], '%m/%d/%Y %H:%M:%S.%f')
    t0 = datetime.datetime.strptime(sterlingTimes[temp], '%m/%d/%Y %H:%M:%S.%f')
    t2 = t0
    if sterling['Side'][i] == 'S':
        side = 'S'
    else:
        side = 'B'
    
    # Format symbol string if length is greater than 6
    if len(symbol) > 6:
        spacedSymbol = str(symbol[:indx]) + "20" + str(symbol[indx + 1:indx + 7]) + str(sterling['Put/Call'][i]) + "{:.3f}".format(
            sterling['Strike Price'][i])
        curPosition = positionDict.setdefault(tradr, {}).setdefault(spacedSymbol, 0)
    else:
        curPosition = positionDict.setdefault(tradr, {}).setdefault(symbol, 0)
        
    # Check if trade is a "spoof"
    spoofFlag = False
    tempDict = {}
    while (t2 - t1) < datetime.timedelta(seconds=2):
        symbolT = sterling['Symbol'][temp]
        accountT = sterling['ID'][temp]
        tradrT = accountT[2:6]
        indxT = symbolT.find(' ')
        amtT = sterling['Exe Qty'][temp]
        spacedSymbolT = symbolT
        if sterling['Side'][temp] == 'S':
            sideT = 'S'
        else:
            sideT = 'B'
        if (len(symbolT) > 6):
            spacedSymbolT = str(symbolT[:indx]) + "20" + str(symbolT[indx + 1:indx + 7]) + str(sterling['Put/Call'][temp])

start = datetime.datetime.now()

for i in range(1, sterLength - 1):
    if sterling['Exe Qty'][i] < 1:
        continue
    temp = i + 1
    if '#' in sterlingTimes[temp]:
        continue
    symbol = sterling['Symbol'][i]
    account = sterling['ID'][i]
    tradr = account[2:6]
    indx = symbol.find(' ')
    amt = sterling['Exe Qty'][i]
    spacedSymbol = symbol
    t1 = datetime.datetime.strptime(sterlingTimes[i], '%m/%d/%Y %H:%M:%S.%f')
    t0 = datetime.datetime.strptime(sterlingTimes[temp], '%m/%d/%Y %H:%M:%S.%f')
    t2 = t0
    if sterling['Side'][i] == 'S':
        side = 'S'
    else:
        side = 'B'
    if (len(symbol) > 6):
        spacedSymbol = str(symbol[:indx]) + "20" + str(symbol[indx + 1:indx + 7]) + str(sterling['Put/Call'][i]) + "{:.3f}".format(
            sterling['Strike Price'][i])
        curPosition = positionDict.setdefault(tradr, {}).setdefault(spacedSymbol, 0)
    else:
        curPosition = positionDict.setdefault(tradr, {}).setdefault(symbol, 0)
    spoofFlag = False
    tempDict = {}
    while (t2 - t1) < datetime.timedelta(seconds=2):
        symbolT = sterling['Symbol'][temp]
        accountT = sterling['ID'][temp]
        tradrT = accountT[2:6]
        indxT = symbolT.find(' ')
        amtT = sterling['Exe Qty'][temp]
        spacedSymbolT = symbolT
        if sterling['Side'][temp] == 'S':
            sideT = 'S'
        else:
            sideT = 'B'
        if (len(symbolT) > 6):
            spacedSymbolT = str(symbolT[:indxT]) + "20" + str(symbolT[indxT + 1:indxT + 7]) + str(sterling['Put/Call'][temp]) + "{:.3f}".format(sterling['Strike Price'][temp])
            tempos = positionDict.setdefault(tradrT, {}).setdefault(spacedSymbolT, 0)
            tempPosition = tempDict.setdefault(tradrT, {}).setdefault(spacedSymbolT, tempos)
        else:
            tempos = positionDict.setdefault(tradrT, {}).setdefault(symbolT, 0)
            tempPosition = tempDict.setdefault(tradrT, {}).setdefault(symbolT, tempos)
        
        if "CANCELED" in sterling['Log'][temp]:
            if not spoofFlag:
                spoofFlag = True
                spIt = 0
                spoofs.loc[spIt, "Date"] = d.date()
                spoofs.loc[spIt, "Time"] = t1.strftime('%H:%M:%S.%f')
                spoofs.loc[spIt, "Time Entered"] = "N/A"
                spoofs.loc[spIt, "Side"] = side
                spoofs.loc[spIt, "Symbol"] = symbol
                spoofs.loc[spIt, "Quantity"] = amt
                spoofs.loc[spIt, "Maturity"] = sterling['Maturity'][i]
                spoofs.loc[spIt, "Strike Price"] = sterling['Strike Price'][i]
                spoofs.loc[spIt, "Order Price"] = sterling['Order Price'][i]
                spoofs.loc[spIt, "Open Close"] = sterling['Open/Close'][i]
                spoofs.loc[spIt, "Order Type"] = sterling['Instrument'][i]
                spoofs.loc[spIt, "Status"] = sterling['Status'][i]
                spoofs.loc[spIt, "OrderID"] = sterling['Exch Cl Ord ID'][i]
                spoofs.loc[spIt, "Exchange"] = sterling['Destination'][i]
                spoofs.loc[spIt, "Account"] = account
                spoofs.loc[spIt, "Position"] = curPosition
                iSpIt = spIt
                spIt += 1
                if (len(symbol) > 6):
                    tempDict.setdefault(tradr, {}).setdefault(spacedSymbol, 0)
                else:
                    tempDict.setdefault(tradr, {}).setdefault(symbol, 0)
                if side == 'S':
                    tempDict[tradr][spacedSymbol] -= amt
                else:
                    tempDict[tradr][spacedSymbol] += amt

            symbol = sterling['Symbol'][temp]
            spoofs.loc[spIt, "Date"] = d.date()
            spoofs.loc[spIt, "Time"] = t2.strftime('%H:%M:%S.%f')
            spoofs.loc[spIt, "Time Entered"] = findEntry(sterling['Exch Cl Ord ID'][temp], temp)
            spoofs.loc[spIt, "Side"] = sterling['Side'][temp]
            spoofs.loc[spIt, "Symbol"] = symbol
            spoofs.loc[spIt, "Quantity"] = sterling['Order Quantity'][temp]
            spoofs.loc[spIt, "Maturity"] = sterling['Maturity'][temp]
            spoofs.loc[spIt, "Strike Price"] = sterling['Strike Price'][temp]
            spoofs.loc[spIt, "Order Price"] = sterling['Order Price'][temp]
            spoofs.loc[spIt, "Order Type"] = sterling['Instrument'][temp]
            spoofs.loc[spIt, "Status"] = sterling['Status'][temp]
            spoofs.loc[spIt, "OrderID"] = sterling['Exch Cl Ord ID'][temp]
            spoofs.loc[spIt, "Exchange"] = sterling['Destination'][temp]
            account = sterling['ID'][temp]
            spoofs.loc[spIt, "Account"] = account
            spoofs.loc[spIt, "Open Close"] = sterling['Open/Close'][temp]
            spoofs.loc[spIt, "Position"] = tempDict[tradrT][spacedSymbolT]

            spIt += 1
        temp += 1


KeyError: 'Time'